In [1]:
import gpt_2_simple as gpt2
import os
import re
from datetime import datetime 
import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm import tqdm
from google.cloud import storage
os.chdir('..')

In [2]:
#os.getcwd()

In [3]:
#!ls

In [4]:
2+2

4

In [5]:
# train - validation - test split
#data = pd.read_csv("data/songs_data_tokenized2.csv", encoding='utf-8')
#N = len(data)
#X = np.array(list(range(N)))
#np.random.shuffle(X)

#train = data.loc[X[:int(.8*N)]]
#validation = data.loc[X[int(.8*N):int(.9*N)]]
#test = data.loc[X[int(.9*N):]]

#train.to_csv("data/train.csv", index=False)
#validation.to_csv("data/validation.csv", index=False)
#test.to_csv("data/test.csv", index=False)

train = pd.read_csv("data/train.csv", encoding='utf-8')
validation = pd.read_csv("data/validation.csv", encoding='utf-8')

In [6]:
model_name="124M"
checkpoint_dir = 'checkpoint'
file_name = "data/train.csv"
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
model_dir = "models/gpt2_validation"
bucket_name = "poem-generator-checkpoints"
run_name='disco_polo_gen_gpt2_validation'

In [7]:
if not os.path.exists(os.path.join(model_dir, model_name)):
    gpt2.download_gpt2(model_dir=model_dir, model_name=model_name)

In [8]:
def get_logits(sess,
             run_name='run1',
             checkpoint_dir='checkpoint',
             model_name=None,
             model_dir='models',
             prefix="<|endoftext|>",
             all=False):

    import tarfile
    import os
    import json
    import requests
    import sys
    import shutil
    import re
    from tqdm import tqdm, trange
    import numpy as np
    import tensorflow as tf
    from tensorflow.core.protobuf import rewriter_config_pb2
    from tensorflow.python.client import device_lib
    import time
    from datetime import datetime
    import csv
    import argparse
    from gpt_2_simple.src import model, sample, encoder, memory_saving_gradients
    from gpt_2_simple.src.load_dataset import load_dataset, Sampler
    from gpt_2_simple.src.accumulate import AccumulatingOptimizer
    
    batch_size=1

    if model_name:
        checkpoint_path = os.path.join(model_dir, model_name)
    else:
        checkpoint_path = os.path.join(checkpoint_dir, run_name)

    enc = encoder.get_encoder(checkpoint_path)
    hparams = model.default_hparams()
    with open(os.path.join(checkpoint_path, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if prefix:
        context = tf.compat.v1.placeholder(tf.int32, [batch_size, None])
        context_tokens = enc.encode(prefix)

    def step(hparams, tokens, past=None):
        lm_output = model.model(hparams=hparams, X=tokens,
                                past=past, reuse=tf.compat.v1.AUTO_REUSE)

        logits = lm_output['logits'][:, :, :hparams.n_vocab]
        presents = lm_output['present']
        presents.set_shape(model.past_shape(
            hparams=hparams, batch_size=batch_size))
        return {
            'logits': logits,
            'presents': presents,
        }

    output = step(hparams, context)

    out = sess.run(output, feed_dict={
                    context: batch_size * [context_tokens]
                })

    if all:
        return out['logits'][0, :, :]  # all logits starting from the second token, n logits for n tokens
    return out['logits'][0, -1, :]  # logits for next token


def get_perplexity(sess,
               run_name='run1',
               checkpoint_dir='checkpoint',
               model_name=None,
               model_dir='models',
               prefix="<|endoftext|>",
               continuation="Hello"):
    
    import tarfile
    import os
    import json
    import requests
    import sys
    import shutil
    import re
    from tqdm import tqdm, trange
    import numpy as np
    import tensorflow as tf
    from tensorflow.core.protobuf import rewriter_config_pb2
    from tensorflow.python.client import device_lib
    import time
    from datetime import datetime
    import csv
    import argparse
    from gpt_2_simple.src import model, sample, encoder, memory_saving_gradients
    from gpt_2_simple.src.load_dataset import load_dataset, Sampler
    from gpt_2_simple.src.accumulate import AccumulatingOptimizer
    
    """
    Returns perplexity score for given continuation of a given prefix.
    
    Examples:
    perplexity(sess, model_name="124M", prefix="Hello, my name is", continuation=" James Smith, I am an engineer")  # returns 17.3124
    perplexity(sess, model_name="124M", prefix="Hello, my name is", continuation=" very else whatever general cat meow.")  # returns 5197.99
    """

    batch_size=1

    if model_name:
        checkpoint_path = os.path.join(model_dir, model_name)
    else:
        checkpoint_path = os.path.join(checkpoint_dir, run_name)

    enc = encoder.get_encoder(checkpoint_path)

    context_tokens = enc.encode(prefix)
    
    context_size = len(context_tokens)
    continuation_tokens = enc.encode(continuation)
    # my line
    cutting_point = min(len(continuation_tokens), 1024-context_size-1)
    continuation_tokens = continuation_tokens[:cutting_point]
    
    full_sentence = prefix+continuation

    logits = get_logits(sess, run_name, checkpoint_dir, model_name, model_dir, full_sentence, all=True)

    logits = logits[context_size-1:-1, :]  # only continuation logits
    logitmeans = np.mean(logits, axis=1)
    logits = logits - logitmeans[:, None]
    explogits = np.exp(logits)
    probabs = explogits / np.sum(explogits,axis=1)[:, None]
    
    probab_scores = np.nan_to_num([probabs[i, index] for i, index in enumerate(continuation_tokens)])
    perplexity = 2 ** (-np.mean(np.log2(probab_scores)))
    return perplexity


def generate_song_part(context, truncation, common_params_dict, savepath=None, min_length_stanza_chars=100,
                       min_length_song_chars=500, chorus_needed=False):
    
    res = gpt2.generate(sess,
              prefix=context,
              truncate=truncation,
              **common_params_dict
              )
    
    good_results = []
    for st in res:
        print(f"Length of generated chunk (chars) = {len(st) - len(context)}, (context len = {len(context)})")
        if len(st) - len(context) > min_length_stanza_chars:# and st.count("<RBEG>") == 0:
            st = st.replace("<RBEG>", "").replace("<EOST>", "")
            good_results.append(st)
            print(st)
    print(f"Out of {len(res)} initial stanzas, {len(good_results)} were left.")
          
    save_count = 0
    to_continue = []
    if savepath:
        for song in good_results:
            if len(song) > min_length_song_chars:
                if savepath == "return":
                    return song
                else:
                    song_name = "sample_" + str(random.randrange(1_000_000_000))
                    with open(savepath+song_name, "w+") as file:
                        file.write(song)
                        save_count += 1
            else:
                to_continue.append(song)
        print(f"{save_count} songs were saved.")
               
            
    for song in to_continue:
        if chorus_needed:
            context = song + "<RBEG>"
            truncation = "<EOST>"
            chorus_needed = False
        else:
            context = song
            truncation = "<EOST>"
        
        generate_song_part(context=context, truncation=truncation, common_params_dict=common_params_dict,
                           savepath="./samples/", min_length_stanza_chars=min_length_stanza_chars,
                           min_length_song_chars=min_length_song_chars, chorus_needed=chorus_needed)
    

class Tee(object):
    def __init__(self, name, mode, encoding="utf-8"):
        self.file = open(name, mode, encoding=encoding)
        self.stdout = sys.stdout
        sys.stdout = self
    def __del__(self):
        sys.stdout = self.stdout
        self.file.close()
    def write(self, data):
        self.file.write(data)
        try:
            self.stdout.write(data)
        except UnicodeEncodeError as err:
            self.stdout.write(f"Writing log didn't succeed due to {err}.")

    def flush(self):
        self.file.flush()

gpt2.get_perplexity = get_perplexity
gpt2.get_logits = get_logits

In [9]:
# gpt2.get_perplexity(sess,
#                run_name=run_name,
#                checkpoint_dir='checkpoint',
#                model_name="124M",
#                model_dir=model_dir,
#                prefix=prefix,
#                continuation="generated")



common_params_dict = {"run_name":run_name, "length":100, "temperature":0.9,
                      "top_p":0.95, "return_as_list":True, "nsamples":1, "batch_size":1}

In [10]:
common_params_dict

{'run_name': 'disco_polo_gen_gpt2_validation',
 'length': 100,
 'temperature': 0.9,
 'top_p': 0.95,
 'return_as_list': True,
 'nsamples': 1,
 'batch_size': 1}

In [11]:
import sys
sys.stdout = Tee("log_data2.dat", mode="a", encoding='utf-8')

Loading checkpoint checkpoint/disco_polo_gen_gpt2_validation/model-1400
INFO:tensorflow:Restoring parameters from checkpoint/disco_polo_gen_gpt2_validation/model-1400
Loading dataset...
dataset has 3092002 tokens
Training...
Saving checkpoint/disco_polo_gen_gpt2_validation/model-1400
INFO:tensorflow:checkpoint/disco_polo_gen_gpt2_validation/model-1400.data-00000-of-00001
INFO:tensorflow:497800
INFO:tensorflow:checkpoint/disco_polo_gen_gpt2_validation/model-1400.meta
INFO:tensorflow:500900
INFO:tensorflow:checkpoint/disco_polo_gen_gpt2_validation/model-1400.index
INFO:tensorflow:500900
======== SAMPLE 1 ========
askie głębi
Miesię, kiedy do mnie, dzisiaj wielki zatańczą
Tylko mi jest sama ją mnie,
Jak wielki mam mieć
<RBEG>Ref.
<REND><REND><REND>
<EOST>
Hej hej, kiedy miała sama ją mnie,
Miała sama ją mnie,
Hej hej hej, kiedy świat mnie mą
Łez tylko zatrzymała się skrycie
Wielka mi mioła i ło żalny i łóży
I choć nieracji mych złów, jak największe łży
<EOST>
<RBEG>Ref.
<REND><REND><REND>

In [12]:
#print("Testing saving output...")

In [ ]:

#for song in validation.song:
#        splitted = song.split("\n")
        #print(splitted)
#        context = splitted[0]
#        del splitted[0]
#        continuation = "".join(splitted)
        #print(song)
        #print(continuation)
        #print(context)
sess = gpt2.start_tf_sess() 

gpt2.finetune(sess,
                  model_dir=model_dir,
                  dataset=file_name,
                  model_name=model_name,
                  reuse=False,
                  steps=8000,
                  run_name=run_name,
                  print_every=10,
                  sample_every=200,
                  save_every=500, 
                  only_train_transformer_layers=False,
                  accumulate_gradients = 1,
                  overwrite=True,
                  restore_from="latest"
                  )

2022-05-26 07:19:22.092399: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-26 07:19:22.168660: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-05-26 07:19:22.168716: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-26 07:19:22.168751: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pytorch-1-10-20220326-144724): /proc/driver/nvidia/version does not exis

In [14]:
sess = gpt2.start_tf_sess() 
steps = 200
scores = []
n_steps = []
i = 0

while i < 3 or (scores[-1] + scores[-2])/2 < scores[-3]:
    tf.compat.v1.reset_default_graph()
    sess = gpt2.start_tf_sess()
    # run model for 200 steps
    gpt2.finetune(sess,
                  model_dir=model_dir,
                  dataset=file_name,
                  model_name=model_name,
                  reuse=False,
                  steps=steps,
                  run_name=run_name,
                  print_every=10,
                  sample_every=200,
                  save_every=500, 
                  only_train_transformer_layers=False,
                  accumulate_gradients = 1,
                  overwrite=True,
                  restore_from="latest"
                  )
    
    # calculate perplexity on test set
    print("Calculating perplexity...")
    tmp_perplexity = []
    ids = np.random.choice(range(len(validation.song)), size=100, replace=False)
    sampled_100_songs = validation.song.loc[ids]
    for song in tqdm(sampled_100_songs):
        splitted = song.split("\n")
        context = splitted[0]
        first_eost = song.find("<EOST>")
        if first_eost < len(context):
            first_eost = min(2 * len(context), len(song))
            
        continuation = song[len(context):first_eost]
        
        #sample = generate_song_part(context=context, truncation="<RBEG>", common_params_dict=common_params_dict,
        #           savepath="return", min_length_stanza_chars=40, min_length_song_chars=700,
        #           chorus_needed=True)
        #print("Calculating perplexity...")
        perp_i = 0
        try:
            perp_i = gpt2.get_perplexity(sess,
                       run_name=run_name,
                       checkpoint_dir='checkpoint',
                       model_name="124M",
                       model_dir=model_dir,
                       prefix=context,
                       continuation=continuation)
        except  Exception as e:
            print(e)
        
        tmp_perplexity.append(perp_i)
        
    perp = sum(tmp_perplexity) / len(validation)
    scores.append(perp)
    print(f"Current perplexity = {perp}")
    # update iterator for stop condition
    i += 1
    
print("Training done.")
print(f"We did {(i+1) * 200} steps in total.")
print("Perplexity scores:")
print(scores)



with open("./validation_info.txt", "w+") as f:
    f.write(f"We did {(i+1) * 200} steps in total.")
    f.write("Perplexity scores:")
    f.write(scores)

2022-05-25 18:51:51.040688: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-25 18:51:51.114893: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-05-25 18:51:51.114971: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-25 18:51:51.115003: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pytorch-1-10-20220326-144724): /proc/driver/nvidia/version does not exis

ResourceExhaustedError: Graph execution error:

Detected at node 'gradients/model/h10/attn/Max_grad/Cast' defined at (most recent call last):
    File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/opt/conda/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/lib/python3.7/asyncio/base_events.py", line 541, in run_forever
      self._run_once()
    File "/opt/conda/lib/python3.